<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Spark typical operations</p><br>
                                


Initializing Spark using pyspark.sql is so easy.

Remember Spark Session is the window to the Spark World

In [1]:
from pyspark.sql import SparkSession
import pandas as pd

pd.pandas.set_option("display.max_columns", None)

sc = SparkSession.builder.appName('Hello').getOrCreate()

data_frame = sc.read.csv('house_regression_train.csv', header=True, inferSchema=True)
data_frame.select('MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street').show(10)

+----------+--------+-----------+-------+------+
|MSSubClass|MSZoning|LotFrontage|LotArea|Street|
+----------+--------+-----------+-------+------+
|        60|      RL|         65|   8450|  Pave|
|        20|      RL|         80|   9600|  Pave|
|        60|      RL|         68|  11250|  Pave|
|        70|      RL|         60|   9550|  Pave|
|        60|      RL|         84|  14260|  Pave|
|        50|      RL|         85|  14115|  Pave|
|        20|      RL|         75|  10084|  Pave|
|        60|      RL|         NA|  10382|  Pave|
|        50|      RM|         51|   6120|  Pave|
|       190|      RL|         50|   7420|  Pave|
+----------+--------+-----------+-------+------+
only showing top 10 rows



Printing schema 

In [2]:
print(type(data_frame))
data_frame.printSchema()

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- 

Selecting specific columns using the traditional SQL select command


In [3]:
data_frame.select(data_frame['SalePrice'], data_frame['OverallQual']*10).orderBy(data_frame.SalePrice.desc()).show(11)

+---------+------------------+
|SalePrice|(OverallQual * 10)|
+---------+------------------+
|   755000|               100|
|   745000|               100|
|   625000|               100|
|   611657|                90|
|   582933|                90|
|   556581|                90|
|   555000|               100|
|   538000|                80|
|   501837|                90|
|   485000|                90|
|   475000|               100|
+---------+------------------+
only showing top 11 rows



#### Count by group
If you want to count the number of occurances by group you can use the count() and groupBy() functions as chained functions.


In [4]:
data_frame.groupBy('OverallQual').count().show()

+-----------+-----+
|OverallQual|count|
+-----------+-----+
|          1|    2|
|          6|  374|
|          3|   20|
|          5|  397|
|          9|   43|
|          4|  116|
|          8|  168|
|          7|  319|
|         10|   18|
|          2|    3|
+-----------+-----+



You can order the data by frequency using the sort function and setting ascending to False

In [5]:
data_frame.groupBy('OverallQual').count().sort('count', ascending=False).show()

+-----------+-----+
|OverallQual|count|
+-----------+-----+
|          5|  397|
|          6|  374|
|          7|  319|
|          8|  168|
|          4|  116|
|          9|   43|
|          3|   20|
|         10|   18|
|          2|    3|
|          1|    2|
+-----------+-----+



#### Describe the data
You can get the summary of the data using the describe function. 

Similar to pandas you will get count, mean, standard deviation, min and max

for illustrative purposes I'm just using a few features from the large dataset

In [6]:
data_frame.select('Electrical','SalePrice', 'LotArea', 'LotFrontage').describe().show()

+-------+----------+------------------+------------------+-----------------+
|summary|Electrical|         SalePrice|           LotArea|      LotFrontage|
+-------+----------+------------------+------------------+-----------------+
|  count|      1460|              1460|              1460|             1460|
|   mean|      null|180921.19589041095|10516.828082191782|70.04995836802665|
| stddev|      null| 79442.50288288663|  9981.26493237915|24.28475177448321|
|    min|     FuseA|             34900|              1300|              100|
|    max|     SBrkr|            755000|            215245|               NA|
+-------+----------+------------------+------------------+-----------------+



To get the summary of one column like the target for example

In [7]:
data_frame.select('SalePrice').describe().show()

+-------+------------------+
|summary|         SalePrice|
+-------+------------------+
|  count|              1460|
|   mean|180921.19589041095|
| stddev| 79442.50288288663|
|    min|             34900|
|    max|            755000|
+-------+------------------+



#### Crosstab  computation
In some occasion it may be interesting to see the descriptive statistics between two pairwise columns.

In [8]:
data_frame.groupBy('GarageQual').count().show()

+----------+-----+
|GarageQual|count|
+----------+-----+
|        Gd|   14|
|        NA|   81|
|        Po|    3|
|        Ex|    3|
|        Fa|   48|
|        TA| 1311|
+----------+-----+



In [9]:
data_frame.crosstab('OverallQual', 'GarageQual').sort('OverallQual_GarageQual').show()

+----------------------+---+---+---+---+---+---+
|OverallQual_GarageQual| Ex| Fa| Gd| NA| Po| TA|
+----------------------+---+---+---+---+---+---+
|                     1|  0|  0|  0|  2|  0|  0|
|                    10|  1|  0|  1|  0|  0| 16|
|                     2|  0|  0|  0|  1|  0|  2|
|                     3|  0|  2|  0|  6|  0| 12|
|                     4|  0| 10|  0| 26|  0| 80|
|                     5|  1| 19|  3| 32|  2|340|
|                     6|  1| 12|  4| 12|  1|344|
|                     7|  0|  5|  3|  1|  0|310|
|                     8|  0|  0|  3|  1|  0|164|
|                     9|  0|  0|  0|  0|  0| 43|
+----------------------+---+---+---+---+---+---+



In [10]:
data_frame.crosstab('GarageQual', 'OverallQual').sort('GarageQual_OverallQual').show()

+----------------------+---+---+---+---+---+---+---+---+---+---+
|GarageQual_OverallQual|  1| 10|  2|  3|  4|  5|  6|  7|  8|  9|
+----------------------+---+---+---+---+---+---+---+---+---+---+
|                    Ex|  0|  1|  0|  0|  0|  1|  1|  0|  0|  0|
|                    Fa|  0|  0|  0|  2| 10| 19| 12|  5|  0|  0|
|                    Gd|  0|  1|  0|  0|  0|  3|  4|  3|  3|  0|
|                    NA|  2|  0|  1|  6| 26| 32| 12|  1|  1|  0|
|                    Po|  0|  0|  0|  0|  0|  2|  1|  0|  0|  0|
|                    TA|  0| 16|  2| 12| 80|340|344|310|164| 43|
+----------------------+---+---+---+---+---+---+---+---+---+---+



#### Drop 
There are two intuitive API calls: drop('column_name').columns to drop a column and dropna() for null values.

remember, the above operation returns a new data frame so you need to set a new variable for it

In [18]:
df = data_frame.drop('Id','MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MiscVal',)
df.columns

['MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice']

#### Filter
You can use the filter function for obvious purposes

In [20]:
df.filter(df.SalePrice>=180921).show(10)

+------+------+--------+-------------+---------+
|MoSold|YrSold|SaleType|SaleCondition|SalePrice|
+------+------+--------+-------------+---------+
|     2|  2008|      WD|       Normal|   208500|
|     5|  2007|      WD|       Normal|   181500|
|     9|  2008|      WD|       Normal|   223500|
|    12|  2008|      WD|       Normal|   250000|
|     8|  2007|      WD|       Normal|   307000|
|    11|  2009|      WD|       Normal|   200000|
|     7|  2006|     New|      Partial|   345000|
|     8|  2007|     New|      Partial|   279500|
|    11|  2006|     New|      Partial|   325300|
|     9|  2008|      WD|       Normal|   230000|
+------+------+--------+-------------+---------+
only showing top 10 rows



#### Descriptive statistics by group
Finally, you group data by groups and compute statistical operations like the mean

In [23]:
data_frame.groupBy('OverallQual').agg({'SalePrice': 'mean'}).sort('OverallQual').show()

+-----------+------------------+
|OverallQual|    avg(SalePrice)|
+-----------+------------------+
|          1|           50150.0|
|          2|51770.333333333336|
|          3|          87473.75|
|          4| 108420.6551724138|
|          5|133523.34760705288|
|          6| 161603.0347593583|
|          7|207716.42319749217|
|          8|274735.53571428574|
|          9| 367513.0232558139|
|         10| 438588.3888888889|
+-----------+------------------+



To convert a spark dataframe to  Pandas


In [24]:
import pandas as pd

sliced_credit = df.toPandas()
type(sliced_credit)

pandas.core.frame.DataFrame

In [25]:
data_frame.select('OverallQual', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice').orderBy('OverallQual').show(11)

+-----------+------+------+--------+-------------+---------+
|OverallQual|MoSold|YrSold|SaleType|SaleCondition|SalePrice|
+-----------+------+------+--------+-------------+---------+
|          1|     1|  2007|      WD|       Normal|    39300|
|          1|     3|  2009|      WD|       Normal|    61000|
|          2|     1|  2009|   ConLw|       Normal|    60000|
|          2|    10|  2006|      WD|      Abnorml|    35311|
|          2|     1|  2009|      WD|       Normal|    60000|
|          3|     5|  2006|      WD|       Normal|    87500|
|          3|    10|  2009|   ConLD|      Abnorml|    85000|
|          3|     6|  2007|      WD|       Normal|   120000|
|          3|     5|  2009|      WD|      Abnorml|    37900|
|          3|     9|  2008|      WD|       Normal|    67000|
|          3|     7|  2008|      WD|       Normal|    52000|
+-----------+------+------+--------+-------------+---------+
only showing top 11 rows



#### SQL Direct
To run SQL queries directly we need to create an SQL temporary view. 

In [33]:
data_frame.createOrReplaceTempView('oganesson')
sql_type = sc.sql('SELECT OverallQual, MoSold , YrSold, SaleType, SaleCondition, SalePrice FROM oganesson WHERE MoSold > 5 ORDER BY SalePrice')
print(type(sql_type))
sql_type.show()


<class 'pyspark.sql.dataframe.DataFrame'>
+-----------+------+------+--------+-------------+---------+
|OverallQual|MoSold|YrSold|SaleType|SaleCondition|SalePrice|
+-----------+------+------+--------+-------------+---------+
|          4|    11|  2009|      WD|      Abnorml|    34900|
|          2|    10|  2006|      WD|      Abnorml|    35311|
|          4|     7|  2008|      WD|       Normal|    40000|
|          3|     7|  2008|      WD|       Normal|    52000|
|          4|     7|  2010|      WD|       Normal|    55000|
|          5|     6|  2010|      WD|       Alloca|    55993|
|          3|     6|  2010|      WD|       Normal|    58500|
|          5|     6|  2009|     COD|      Abnorml|    60000|
|          5|    12|  2009|      WD|       Normal|    62383|
|          4|     8|  2007|      WD|       Normal|    64500|
|          5|     8|  2009|      WD|      Abnorml|    66500|
|          5|    11|  2007|      WD|      Abnorml|    67000|
|          3|     9|  2008|      WD|       

#### Converting an sql dataframe to a pandas for downstream operations

In [34]:
sql_changed = sql_type.toPandas()
type(sql_changed)

pandas.core.frame.DataFrame

If you need an RDD, you can use the command below to convert a dataframe to an RDD

In [35]:
rdd1 = data_frame.rdd
type(rdd1)

pyspark.rdd.RDD

In [36]:
rdd1.take(5)

[Row(Id=1, MSSubClass=60, MSZoning='RL', LotFrontage='65', LotArea=8450, Street='Pave', Alley='NA', LotShape='Reg', LandContour='Lvl', Utilities='AllPub', LotConfig='Inside', LandSlope='Gtl', Neighborhood='CollgCr', Condition1='Norm', Condition2='Norm', BldgType='1Fam', HouseStyle='2Story', OverallQual=7, OverallCond=5, YearBuilt=2003, YearRemodAdd=2003, RoofStyle='Gable', RoofMatl='CompShg', Exterior1st='VinylSd', Exterior2nd='VinylSd', MasVnrType='BrkFace', MasVnrArea='196', ExterQual='Gd', ExterCond='TA', Foundation='PConc', BsmtQual='Gd', BsmtCond='TA', BsmtExposure='No', BsmtFinType1='GLQ', BsmtFinSF1=706, BsmtFinType2='Unf', BsmtFinSF2=0, BsmtUnfSF=150, TotalBsmtSF=856, Heating='GasA', HeatingQC='Ex', CentralAir='Y', Electrical='SBrkr', 1stFlrSF=856, 2ndFlrSF=854, LowQualFinSF=0, GrLivArea=1710, BsmtFullBath=1, BsmtHalfBath=0, FullBath=2, HalfBath=1, BedroomAbvGr=3, KitchenAbvGr=1, KitchenQual='Gd', TotRmsAbvGrd=8, Functional='Typ', Fireplaces=0, FireplaceQu='NA', GarageType='Att

In [37]:
rdd1.count()


1460

### Persisting data
There are several persit options available in Pyspark
Let us see them one at a time:

##### 1.) MEMORY_ONLY:
In this storage level the whole of the RDD is stored as deserialized Java object in the JVM. It tries to use all the memory available for caching so it does not have to use the disk. If the memory is insufficient it will not cache some of the partitions and those will be recomputed when needed. In this mode the storage utilization is maximum and CPU computation is least.
##### 2.) MEMORY_AND_DISK:
In this storage level the RDD is stored in memory and only the excess partitions are spilled to the disk. When needed again the partitions are read from the disk and served. In this storage level, the memory utilization is maximum along with some disk and CPU utilization is medium.
###### 3.) MEMORY_ONLY_SER:
In this level Spark stores the RDD as a serialized Java object, one byte-array per partition. It is very much optimized for space compared to deserialized Java object, especially in case of fast serializer. But this makes CPU utilization very high and it does not use the disk at all.
###### 4.) MEMORY_AND_DISK_SER: 
This level is very similar to MEMORY_ONLY_SER with the only difference that the excess partitions which do not fit in memory are written to disk. So they are read from disk when needed, instead of recompute being triggered. The space used for storage is low but CPU computation time increases.
###### 5.)DISK_ONLY: 
In this level, the RDD is persisted on disk only and nothing in memory. So the memory utilization is minimal but the CPU computation time increases a lot.
######  6MEMORY_ONLY_2 and MEMORY_AND_DISK_2:
These are similar to MEMORY_ ONLY and MEMORY_ AND_DISK. The only difference is that each partition of the RDD is replicated on two nodes on the cluster.

In [38]:
from pyspark import StorageLevel

In [39]:
rdd1.persist(storageLevel=StorageLevel.MEMORY_AND_DISK)

MapPartitionsRDD[148] at javaToPython at NativeMethodAccessorImpl.java:0

It is a good idea to always release the memory held by persist above.

In [40]:
rdd1.unpersist()

MapPartitionsRDD[148] at javaToPython at NativeMethodAccessorImpl.java:0